# Coronavirus - Gráfica
## A partir de los datos de EU Open Data Portal

Los datos están recogidos de https://data.europa.eu/euodp/en/data/dataset/covid-19-coronavirus-data

Se descarga la versión **CSV** de los datos, que se actualizan a diario.

### Captura de los datos (un minuto más o menos)

In [1]:
## Petición del CSV vía HTTP y volcado en SQLite. Cálculo de campos adicionales ----
## Petición HTTP de los datos ----
##--------------------------------
import http.client as cli

## No olvidar el slash en el request, si no, habrá redirección (que este cliente no procesa) ----
conn = cli.HTTPSConnection('opendata.ecdc.europa.eu')
conn.request("GET", "/covid19/casedistribution/csv/")
r1 = conn.getresponse()
#r1.status, r1.reason

if r1.status == 200:
    print("Leyendo y tratando los datos...")
    data1 = r1.read().decode('utf-8')            ## Importante el decode(). Los datos vienen en binario ----
    
else:
    print(f"No hubo éxito en petición http: {r1.status}-{r1.reason}")
    assert False

## -------------------------------
## Procesado del CSV ----
##--------------------------------
import csv

print("Procesando el CSV...")

campos = None
filas = []

d_open = csv.reader(data1.splitlines())       ## El reader recibe un array de líneas ----
for n, line in enumerate(d_open):
    if n == 0:
        campos = line                         ## Guardamos los nombres de los campos ----
        #print(campos)
    else:
        filas.append(line)                    ## Y el array de valores, línea a línea ----
        
##------------------------------------------
## Comandos SQL de creación e inserción ----
##------------------------------------------

create_sql = "CREATE TABLE covid (\n" + \
', \n'.join(
    
    [c + (' INTEGER' if c in ['day', 'month', 'year', 'cases', 'deaths', 'popData2018'] else ' TEXT') 
     for c in campos]

) + \
'\n);'

#print(create_sql)

insert_sql = "INSERT INTO covid VALUES (" + ''.join(['?, ' for x in range(len(campos))])[:-2] + ")"

#print(insert_sql)

##---------------------------------------------
## Creación de la base de datos en memoria ----
##---------------------------------------------
import sqlite3

print("Creando la tabla de datos en memoria...")

conn = sqlite3.connect(":memory:")
cur = conn.cursor()
cur.execute(create_sql)
cur.executemany(insert_sql, filas)

del filas    ## Liberamos la memoria ----

## ----------------------------------------------------------------------
## Añadimos campo de fecha nuevo (dateISO) susceptible de ordenación ----
## ----------------------------------------------------------------------

print("Añadiendo campo dateISO...")

alter_sql = 'ALTER TABLE covid ADD COLUMN dateISO TEXT'
cur.execute(alter_sql)

## Capturamos los datos de fecha para reconstruirlos ----
cur.execute("SELECT DISTINCT dateRep, year, month, day FROM covid")
rows = cur.fetchall()

## Creamos el conjunto de las nuevas fechas formateadas ----
new_date = []
for dateRep, year, month, day in rows:
    dateIso = f"{year}-{month:02}-{day:02}"
    new_date.append([dateIso, dateRep])

## Actualizamos el nuevo campo con las fechas en el nuevo formato yyyy-mm-dd ----
update_sql = "UPDATE covid SET dateISO = ? WHERE dateRep = ?"
cur.executemany(update_sql, new_date)    

## ------------------------------------------------------------------
## Unos pocos países vienen sin población. Las busco manualmente ----
## ------------------------------------------------------------------

print("Actualizando poblaciones que faltan...")

## A raíz de un error, descubro que me faltan estas poblaciones ----
cur.execute("SELECT DISTINCT countriesAndTerritories FROM covid WHERE popdata2018 = '' ORDER BY dateISO")

## Esta parte sirvió en la parte interactiva, para leer los países de más abajo ----
if False:
    rows = cur.fetchall()
    for row in rows:
        print(row)
    
## ------------------------------------------------
## Actualización de las poblaciones que faltan ----
## ------------------------------------------------

## Busqué las poblaciones en https://www.worldometers.info/world-population/population-by-country/ y Wikipedia ----
pop_needed = [
    ('Czechia', 10_708_981),
    ('Anguilla', 15_003),
    ('Falkland_Islands_(Malvinas)', 2.840),
    ('Eritrea', 3_546_421),
    ('Bonaire, Saint Eustatius and Saba', 25_200)
]

## Pero SQLITE necesita los parámetros en el orden que aparecen, así que tengo que invertirlos ----
pop_needed = [[x[1], x[0]] for x in pop_needed]

## Arreglado! ----
cur.executemany('UPDATE covid SET popData2018 = ? where countriesAndTerritories = ?', pop_needed)

del pop_needed    ## Borrado datos auxiliares ----

## ------------------------------------------------------
## Añado otro campo para el acumulado hasta cada día ----
## ------------------------------------------------------

print("Calculando el campo deaths_accum... Esto llevará un rato...")

alter_sql = 'ALTER TABLE covid ADD COLUMN deaths_accum INTEGER' 
cur.execute(alter_sql)

## ---------------------------------------------------------------------
## Necesito una lista de países y días para el cálculo de las
## muertes acumuladas por país y día ----
## ---------------------------------------------------------------------
sql = "SELECT DISTINCT dateISO, geoId FROM covid ORDER BY 1"
cur.execute(sql)
fechas_geo = cur.fetchall()

## -----------------------------------------------------------
## Cálculo de las muertes acumuladas (para cada día, geo) ---- 
## -----------------------------------------------------------

n_total = len(fechas_geo)
print(f'Total : {n_total}')

sql = """
SELECT geoId, dateISO, SUM(deaths)
FROM covid
WHERE dateISO <= ? AND geoId == ?
"""
covid_acum = []
for i, fecha in enumerate(fechas_geo):
    cur.execute(sql, fecha)
    covid_acum += cur.fetchall()
    if i % 1000 == 0:
        print(i, end=" ")
print()

del fechas_geo     ## Limpieza de datos auxiliares ----

## -----------------------------------------------------------
## Añado un índice para agilizar el UPDATE que nos falta  ---- 
## -----------------------------------------------------------

print("Añadiendo el índice para acelerar inserción del campo deaths_accum...")

sql = """CREATE INDEX covid_index
on covid (geoId, dateISO)"""
cur.execute(sql)

## --------------------------------------------------
## Ahora sí, actualizamos el campo deaths_accum  ---- 
## --------------------------------------------------

print("Actualizando el campo deaths_accum...")

sql = "UPDATE covid SET deaths_accum = ? WHERE dateISO = ? AND geoId = ?"
covid_acum = [(x[2], x[1], x[0]) for x in covid_acum]
cur.executemany(sql, covid_acum)
cur.rowcount

del covid_acum     ## Más limpieza de datos auxiliares ----

print("=== FIN DEL PROCESO ===")

Leyendo y tratando los datos...
Procesando el CSV...
Creando la tabla de datos en memoria...
Añadiendo campo dateISO...
Actualizando poblaciones que faltan...
Calculando el campo deaths_accum... Esto llevará un rato...
Total : 11358
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 
Añadiendo el índice para acelerar inserción del campo deaths_accum...
Actualizando el campo deaths_accum...
=== FIN DEL PROCESO ===


### Campos de la tabla covid 


In [2]:
## -----------------------------
cur.execute("SELECT * FROM covid LIMIT 0")
campos_n = [tuple[0] for tuple in cur.description]
print(campos_n)

['dateRep', 'day', 'month', 'year', 'cases', 'deaths', 'countriesAndTerritories', 'geoId', 'countryterritoryCode', 'popData2018', 'dateISO', 'deaths_accum']


## Lista de países ordenados por muertes (decrecientes)

Servirá para poblar la gráfica de ordenadamente

In [3]:
# Creación de la lista "paises", necesria más adelante (muestra sólo 5) ----
cur.execute("""
SELECT geoId, countriesAndTerritories
FROM (
    SELECT geoId, countriesAndTerritories, MIN(dateISO), MAX(deaths_accum)
    FROM covid
    WHERE cases + deaths > 1 
    GROUP BY geoId, countriesAndTerritories
    ORDER BY 4 DESC, 2
) AS a
WHERE LENGTH(geoId) = 2
""")

paises = cur.fetchall()
paises[:5]

[('US', 'United_States_of_America'),
 ('IT', 'Italy'),
 ('ES', 'Spain'),
 ('FR', 'France'),
 ('UK', 'United_Kingdom')]

## Grafica del crecimiento de víctimas

Se muestra los días en el eje X y los países en el eje Y. El tamaño de los marcadores es proporcional a las muertes.

**Lista de parámetros**

* **num_paises**: las filas que se mostrarán
* **cuantos_dias**: los días hacia atrás desde la fecha actual

In [4]:
## Función explorar(), donde se prepara el gráfico cronológico por países ----
import matplotlib.pyplot as plt
from matplotlib import ticker

def explorar(num_paises, cuantos_dias):

    #num_paises = 15
    #cuantos_dias = 60

    ## Invierto y limito la selección de países (va de abajo arriba, y quiero que vaya al revés) ----
    paises_seleccion = paises[:num_paises][::-1]

    fig = None
    fig = plt.figure(figsize=(13, num_paises))    ## El tamaño vertical se ajusta dinámicamente ----
    ax = fig.add_subplot(111)

    last_deaths = []

    for i, (geo, nom) in enumerate(paises_seleccion):
        #print(i+10, geo, end=' ')

        cur.execute('select dateISO, deaths_accum from covid where geoId = ? order by 1', [geo])
        tmp = cur.fetchall()
        #print(len(tmp))
        x = [x[0] for x in tmp]
        y = [i] * len(x)
        s = [x[1]/num_paises for x in tmp]
        last_deaths.append(s[-1] * num_paises)

        ax.scatter(x=x, y=y, s=s, alpha=.35) 

        #if i == 0:
    ax.yaxis.tick_right()
    ax.set_yticks(range(len(paises_seleccion)))
    ax.set_yticklabels([f"{x}\n({y:_.0f})" for x, y in zip([x[1] for x in paises_seleccion], last_deaths)])
    ax.tick_params(labelsize=15)

    plt.xticks(rotation=30)

    #for i, label in enumerate(ax.xaxis.get_ticklabels()[::-1]):
    #    label.set_visible(False)
    #    if i % 7 == 0:
    #        label.set_visible(True)

    xi, xd = plt.xlim()
    plt.xlim(xd - cuantos_dias, xd)
    ax.xaxis.grid(True)
    
    M = int(cuantos_dias / 7)
    xticks = ticker.MaxNLocator(M)

    # Set the yaxis major locator using your ticker object. You can also choose the minor
    # tick positions with set_minor_locator.
    ax.xaxis.set_major_locator(xticks)


    plt.savefig("coronavirus.png")

    pass

In [5]:
## Presentación interactiva vía widgets ----
import ipywidgets as widgets

from ipywidgets import IntSlider, HBox, interact

#sl_num_paises = IntSlider(description='A too long description',)

interact(explorar, num_paises=10, cuantos_dias=60, continuous_update=False)
pass

interactive(children=(IntSlider(value=10, description='num_paises', max=30, min=-10), IntSlider(value=60, desc…

## Consultas frecuentes 

In [6]:
## Función para visualizar tablas ----
def get_table(list2D, column_names):
    s = ''
    for cn in column_names:
        s += f'<th style="text-align: center">{cn}</th>'
    s = '<tr>' + s + '</tr>\n'

    for row in list2D:
        s2 = ''
        for elem in row:
            s2 += f'<td>{elem}</td>'
        s2 = '<tr>' + s2 + '</tr>\n'
        s += s2
        
    return '<table>' + s + '</table>'

In [8]:
## Muertos acumulados por países, en orden decreciente ----
from IPython.display import display, HTML

cur.execute("""
SELECT a.countriesAndTerritories AS paises, 
        SUM(a.cases) AS cases, SUM(a.deaths) AS deaths, AVG(a.popData2018) AS pobl
FROM  covid AS a
GROUP BY a.countriesAndTerritories
ORDER BY deaths DESC""")

rows = cur.fetchall()
columnas = ('País','Casos','Muertos','Población')
filas = ((f"{x[0]:30.30}", f"{x[1]:>7_}", f"{x[2]:>6_}", f"{x[3]:>13_.0f}") for x in rows)

display(HTML(get_table(filas, columnas )))

País,Casos,Muertos,Población
United_States_of_America,671_331,33_284,327_167_434
Italy,168_941,22_172,60_431_283
Spain,182_816,19_130,46_723_749
France,108_847,17_920,66_987_244
United_Kingdom,103_093,13_729,66_488_991
Iran,77_995,4_869,81_800_269
Belgium,34_809,4_857,11_422_068
China,83_754,4_636,1_392_730_000
Germany,133_830,3_868,82_927_922
Netherlands,29_214,3_315,17_231_017


In [9]:
## Muertos en el último día por países, en orden decreciente ----
from IPython.display import display, HTML

cur.execute('''
SELECT countriesAndTerritories AS paises, 
    cases, deaths, popData2018
FROM covid
WHERE dateISO = (SELECT MAX(dateISO) FROM covid)
ORDER BY 3 DESC
''')

rows = cur.fetchall()
columnas = ('País','Casos','Muertos','Población')
filas = ((f"{x[0]:30.30}", f"{x[1]:>7_}", f"{x[2]:>6_}", f"{x[3]:>13_.0f}") for x in rows)

display(HTML(get_table(filas, columnas )))

País,Casos,Muertos,Población
United_States_of_America,31_667,2_299,327_167_434
China,352,1_290,1_392_730_000
United_Kingdom,4_617,861,66_488_991
France,2_641,753,66_987_244
Spain,5_183,551,46_723_749
Italy,3_786,525,60_431_283
Belgium,1_236,417,11_422_068
Switzerland,315,360,8_516_543
Germany,3_380,299,82_927_922
Brazil,2_105,188,209_469_333


In [10]:
# Código para ver evolución casos/muertes por país ----
import ipywidgets as widgets

cur.execute("""
SELECT DISTINCT countriesAndTerritories, geoId
FROM  covid As a
WHERE deaths > 10
ORDER BY countriesAndTerritories""")

wpaises = cur.fetchall()

wcasos = [['Casos', 'c'], ['Muertes', 'm'], ['Ambos', 'a']]

@interact(geoId=wpaises, tipo=wcasos)
def crecimiento(geoId, tipo='a'):
    cur.execute("""
    SELECT dateISO, cases, deaths, countriesAndTerritories
    FROM covid
    WHERE geoId = ?
    AND dateISO >= (SELECT MIN(dateISO) FROM covid WHERE deaths > 0 AND geoId = ?)
    ORDER BY 1
    """, [geoId, geoId])

    historial = cur.fetchall()

    x = [x[0] for x in historial]
    y1 = [x[1] for x in historial]
    y2 = [x[2] for x in historial]
    pais =historial[0][3]

    import matplotlib.pyplot as plt
    #import matplotlib.dates as mdates

    plt.rcParams['figure.figsize'] = [10, 5]

    if tipo == 'c':
        plt.title("Casos de coronavirus en " + pais)
        plt.plot(x, y1)
    elif tipo == 'm':
        plt.title("Muertes por coronavirus en " + pais)
        plt.plot(x, y2)
    else:
        plt.title("Casos y muertes por coronavirus en " + pais)
        plt.plot(x, y1, y2)
        
    plt.xticks(rotation=45)
    plt.grid(True)
    for i, label in enumerate(plt.gca().xaxis.get_ticklabels()[::-1]):
        label.set_visible(False)
        if i % 7 == 0:
            label.set_visible(True)
            
    the_table = plt.table(cellText=[x[:-1] for x in historial[:-14:-1]],
                      #rowLabels=rows,
                      #rowColours=colors,
                      colLabels=('día', 'casos', 'muertos', 'no sé'),
                      loc='right',
                      bbox=[1, 0, .5, 1.])
    plt.show()

    #ii = -1
    #for ii, (d, c, m, _) in enumerate(historial[::-1]):
    #    print(f"{d:10}: {c:>7_} casos, {m:>6_} muertos.")
    #print(ii)


interactive(children=(Dropdown(description='geoId', options=(('Algeria', 'DZ'), ('Argentina', 'AR'), ('Austria…

## Media móvil de 7 días

En este gráfico se muestran los muertos diarios y una média móvil de los últimos siete días.

La media móvil tiene como objeto quedarse con la tendencia, promediando los picos que puedan producirse durante los últimos siete días.

In [11]:
## Muertos y media móvil semanal por país ----
import ipywidgets as widgets

cur.execute("""
SELECT DISTINCT countriesAndTerritories, geoId
FROM  covid As a
WHERE deaths > 10
ORDER BY countriesAndTerritories""")

wpaises = cur.fetchall()

@interact(geoId=wpaises)
def crecimiento2(geoId):
    cur.execute("""

    SELECT
        dateISO, deaths, 
        ROUND(AVG(deaths) OVER (
            ORDER BY dateISO
            ROWS BETWEEN 6 PRECEDING AND 0 FOLLOWING
        ), 2) AS MM7, countriesAndTerritories
    FROM 
        covid
    WHERE geoId = ?
    AND dateISO >= (SELECT MIN(dateISO) FROM covid WHERE deaths > 0 AND geoId = ?)


    """, (geoId, geoId))
    mm7 = cur.fetchall()

    x = [x[0] for x in mm7]
    y = [x[1] for x in mm7]
    y2 = [x[2] for x in mm7]
    pais = mm7[0][3]
    
    import matplotlib.pyplot as plt
    import matplotlib.dates as mdates

    plt.rcParams['figure.figsize'] = [10, 5]

    plt.xticks(rotation=45)
    plt.plot(x,y, y2)
    plt.grid(True)
    for i, label in enumerate(plt.gca().xaxis.get_ticklabels()[::-1]):
        label.set_visible(False)
        if i % 7 == 0:
            label.set_visible(True)
            
    the_table = plt.table(cellText=[x[:-1] for x in mm7[:-14:-1]],
                      #rowLabels=rows,
                      #rowColours=colors,
                      colLabels=('día', 'muertes', 'media', 'no sé'),
                      loc='right',
                      bbox=[1, 0, .5, 1.])

    plt.title("Muertos diarios y media móvil a 7 días en " + pais)
    plt.show()

    #ii = -1
    #for ii, (d, c, m, _) in enumerate(mm7[::-1]):
    #    print(f"{d:10}: {c:>7_} muertos, {m:>8_} media móvil.")
    #print(ii)


interactive(children=(Dropdown(description='geoId', options=(('Algeria', 'DZ'), ('Argentina', 'AR'), ('Austria…

## Países con peor comportamiento

Miramos el crecimiento o decrecimiento de la media móvil de un día para otro.

Ordenamos por valor absoluto de la diferencia, pero también se puede por el porcentaje o por la propia media.

In [13]:
## Última media móvil e incremento de media móvil desde día anterior (por países) ----
from IPython.display import display, HTML

orderby = [('Media 7d', 'MM7'), ('Dif.media ant', 'diff'), ('Dif.med. (%)', 'percent')]

@interact(order_by=orderby)
def get_diff_mm(order_by='diff'):
    cur.execute(f"""
        SELECT
            geoId, pais, dateISO, MM7, diff, (diff/MM7*100) AS percent
        FROM (
            SELECT
                geoId, pais,
                dateISO, MM7,
                    ROUND(MM7 - 
                        LAG(MM7, 1, 0) OVER (
                            PARTITION BY geoId
                            ORDER BY dateISO
                        ), 2) AS diff
            FROM (
                SELECT
                    geoId, MAX(countriesAndTerritories) As pais,
                    dateISO, deaths,
                    ROUND(AVG(deaths) OVER (
                        ORDER BY geoId, dateISO 
                        ROWS BETWEEN 6 PRECEDING AND 0 FOLLOWING
                    ), 2) AS MM7
                FROM 
                    covid
                GROUP BY
                    geoId, dateISO
            ) AS a
        ) AS b
        WHERE dateISO = (SELECT MAX(dateISO) from covid) AND MM7 > 10
        ORDER BY {order_by} DESC
        """)

    diff_mm = cur.fetchall()
    columnas = ('País','Media 7d','Dif.media ant.','Dif.med.(%)')
    filas = (([x[1], f"{x[3]:5_.0f}", f'{x[4]:5_.1f}', f'{x[5]:6_.1f} %']) for x in diff_mm)

    display(HTML(get_table(filas, columnas )))

interactive(children=(Dropdown(description='order_by', index=1, options=(('Media 7d', 'MM7'), ('Dif.media ant'…